In [3]:

import pickle
import scipy.stats
import numpy as np
import matplotlib.pyplot as plt



In [7]:
from keras.models import Model, Sequential
from keras.layers import Conv1D,LSTM,Bidirectional,GRU,Input,Dropout,BatchNormalization,Reshape,Activation


In [ ]:
# load examples

In [10]:
def generate_gaussian(length,mean,std):
    x_values = np.arange(0,length)
    y_values = scipy.stats.norm(mean,std)
    y_values = y_values.pdf(x_values)
    #re-escale to peak at one
    y_values = y_values/np.max(y_values)
    # truncate at 2 sigmas
    y_truncated = np.zeros((30000))
    y_truncated[int(mean-2*std):int(mean+2*std)] = y_values[int(mean-2*std):int(mean+2*std)]
    
    return y_values,y_truncated

def generate_box(index1,index2):
    y_label = np.zeros((30000))
    end = int((index2-index1)*1.4)
    y_label[int(index1):int(index2)+end] = 1
    return y_label

def create_training_example(filename):
    # load the pickle file
    a_file = open(filename,'rb')
    data  = pickle.load(a_file)
    x     = data['data']
    # check if the initial phase is Pg or Pn
    if data['Pn']!=0:
        y = generate_box(data['Pn'],data['Sn'])
    if data['Pg']!=0:
        y = generate_box(data['Pg'],data['Sg'])
    return x,y
    

In [13]:
x1,y1 = create_training_example('data__Pn_100.pkl')

(30000,)

In [ ]:
def model(input_shape):
    
    
    X_input=Input(shape=input_shape)
    
    # A dense block
    X = Conv1D(filters=196,kernel_size=100,stride=5)(X_input)
    X = BatchNormalization()(X)
    X = Activation('relu')(X)
    X = Dropout(rate=0.8)(X)
    
    
    
    